In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      bit=4,
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      bit=4,
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      bit=4,
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
   

In [2]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9316/10000 (93%)



In [3]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
42 -th layer prehooked
47 -th layer prehooked
51 -th layer prehooked
55 -th layer prehooked


In [4]:
weight_q = model.features[3].weight_q
w_alpha = model.features[3].weight_quant.wgt_alpha
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
print(weight_int)

tensor([[[[-1.0000, -1.0000, -0.0000],
          [-1.0000, -0.0000, -1.0000],
          [ 1.0000, -1.0000, -0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 2.0000, -3.0000,  3.0000],
          [ 1.0000, -3.0000,  3.0000],
          [ 1.0000, -2.0000,  0.0000]],

         ...,

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 7.0000,  3.0000, -6.0000],
          [ 7.0000, -3.0000, -3.0000],
          [ 3.0000, -1.0000, -3.0000]],

         [[-0.0000, -1.0000, -0.0000],
          [-0.0000, -1.0000, -1.0000],
          [-0.0000, -1.0000, -1.0000]]],


        [[[-0.0000, -1.0000,  1.0000],
          [ 0.0000, -1.0000,  1.0000],
          [ 1.0000,  1.0000, -1.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 1.0000, -1.0000, -2

In [5]:
act = save_output.outputs[1][0]
act_alpha  = model.features[3].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

act_q = act_quant_fn(act, act_alpha)

act_int = act_q / (act_alpha / (2**act_bit-1))
print(act_int)

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  1.0000,  2.0000],
          [ 0.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000,  1.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  2.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  2.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 6.0000,  9.0000, 10.0000,  ...,  7.0000,  7.0000, 14.0000],
          [ 3.0000,  8.0000, 1

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
conv_int.bias = model.features[3].bias
output_int = conv_int(act_int)
output_recovered = output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))
print(output_recovered)

tensor([[[[-7.0881e-01, -6.8045e+00,  6.3084e+00,  ...,  6.0957e+00,
           -8.7183e+00,  1.8642e+01],
          [-9.2853e+00, -1.2758e+01,  3.1187e+00,  ...,  3.1896e+00,
           -1.5168e+01,  2.7076e+01],
          [-6.3084e+00, -2.0910e+01,  7.0881e-02,  ..., -2.6935e+00,
           -2.1335e+01,  3.1258e+01],
          ...,
          [-4.2528e-01,  5.1672e+01,  2.7147e+01,  ...,  3.4944e+01,
            4.5718e+01, -2.3320e+01],
          [ 4.0402e+00,  3.6433e+01,  2.4737e+01,  ...,  4.7277e+01,
            1.5452e+01,  8.2930e+00],
          [ 4.4655e+00,  2.1902e+01,  1.6870e+01,  ...,  2.6580e+01,
            5.5996e+00,  1.5168e+01]],

         [[-1.1128e+01, -9.3562e+00,  8.0095e+00,  ...,  1.7791e+01,
            1.7649e+01,  1.8783e+01],
          [-1.8075e+01, -2.3958e+01, -7.7969e+00,  ...,  5.3869e+00,
            1.2050e+01,  2.1760e+01],
          [-1.7224e+01, -2.9841e+01, -1.9138e+01,  ..., -1.0349e+01,
            5.2452e+00,  2.1548e+01],
          ...,
     

In [7]:
conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, padding=1)
conv_ref.weight = model.features[3].weight_q
conv_ref.bias = model.features[3].bias
output_ref = conv_ref(act)
print(output_ref)

tensor([[[[-9.7382e-01, -6.3272e+00,  6.6631e+00,  ...,  6.0691e+00,
           -1.5720e+01,  2.0214e+01],
          [-9.3676e+00, -1.2439e+01,  3.0917e+00,  ...,  2.4946e+00,
           -2.6449e+01,  3.0302e+01],
          [-7.0341e+00, -2.2486e+01,  5.0354e-01,  ..., -1.8553e+00,
           -3.4564e+01,  3.5178e+01],
          ...,
          [ 2.0096e+00,  7.7623e+01,  2.6615e+01,  ...,  3.5133e+01,
            4.9417e+01, -2.3573e+01],
          [ 1.3302e+01,  5.6443e+01,  2.4904e+01,  ...,  4.8974e+01,
            1.8005e+01,  8.0739e+00],
          [ 1.4158e+01,  2.9152e+01,  1.8555e+01,  ...,  3.0913e+01,
            7.3132e+00,  1.4851e+01]],

         [[-1.0986e+01, -9.6426e+00,  8.8741e+00,  ...,  1.7223e+01,
            2.1413e+01,  2.3557e+01],
          [-1.7594e+01, -2.3959e+01, -7.9176e+00,  ...,  5.2591e+00,
            1.3552e+01,  2.6867e+01],
          [-1.6513e+01, -2.7878e+01, -1.8428e+01,  ..., -1.0071e+01,
            7.1441e+00,  2.6153e+01],
          ...,
     

In [8]:
# act_int.size = torch.Size([128, 64, 32, 32])  <- batch_size, input_ch, ni, nj
a_int = act_int[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [64, 32, 32]

# conv_int.weight.size() = torch.Size([64, 64, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
# w_int.weight.size() = torch.Size([64, 64, 9])
                      
padding = 1
stride = 1
array_size = 16 # row and column number

ic_tileg = range(int(w_int.size(1) / array_size))  ## input channel tile
oc_tileg = range(int(w_int.size(0) / array_size))  ## output channel tile

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(array_size)  ## input channel 
ocg = range(array_size)  ## output channel

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(weight_int.size(0), len(nig)+padding*2, len(njg)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))  ## mergin ni and nj index into nij
# a_pad.size() = [64, (32+2pad)*(32+2pad)]
a_tile = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, a_pad.size(-1)).cuda()
for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        for ic in icg:
            a_tile[ic_tile, oc_tile, ic, :] = a_pad[ic_tile*array_size+ic, :]

w_tile = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, array_size, len(kijg)).cuda()
for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        for ic in icg:
            for oc in ocg:
                w_tile[ic_tile, oc_tile, ic, oc, :] = w_int[ic_tile*array_size+ic, oc_tile*array_size+oc, :]
                
###########################################



In [9]:
p_nijg = range(a_pad.size(1)) ## paded activation's nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for oc_tile in oc_tileg:
        for ic_tile in ic_tileg:
            m = nn.Linear(array_size, array_size, bias=False).cuda()
            m.weight = torch.nn.Parameter(w_tile[oc_tile, ic_tile, :, :, kij])
            for nij in p_nijg:     # time domain, sequentially given input
                psum[ic_tile, oc_tile, :, nij, kij] += m(a_tile[ic_tile, oc_tile, :, nij]).cuda()


In [10]:
psum.shape

torch.Size([4, 4, 16, 1156, 9])

In [11]:
psum[0,0,10,100:200,0] + psum[1,0,10,100:200,0] + psum[2,0,10,100:200,0] + psum[3,0,10,100:200,0]

tensor([ -54.,    0.,    0.,   22.,  -27.,  -51., -101.,  -76.,  -63.,  -53.,
         -56.,  -53.,  -68.,  -67.,  -35.,   -2.,   -7.,  -14.,  -14.,  -17.,
         -18.,   14.,   38.,   71.,  108.,  117.,   94.,   84.,   62.,    4.,
         -21.,  -28.,  -46.,  -74.,  -53.,    0.,    0.,   33.,  -12.,    7.,
         -50.,  -70.,  -54.,  -54.,  -59.,  -70.,  -70.,  -39.,  -92.,   -5.,
          39.,   46.,   24.,   11.,   -1.,   31.,   50.,   45.,   48.,   89.,
         116.,  121.,  139.,  114.,   57.,    6.,  -30.,  -64.,  -68.,    0.,
           0.,   55.,    5.,   31.,    5.,  -32.,  -48.,  -67.,  -52.,  -50.,
         -59.,  -52., -125.,  -69.,   -3.,   34.,   21.,   28.,   74.,   73.,
         -17.,  -37.,  -39.,  -26.,  -23.,   36.,   58.,  103.,   96.,   53.],
       device='cuda:0', grad_fn=<AddBackward0>)

In [12]:
######## Easier 2D version ########

import math

kig = range(int(math.sqrt(len(kijg))))
kjg = range(int(math.sqrt(len(kijg))))
    
o_nig = range(a_int.size(1))
o_njg = range(a_int.size(2))
    
out = torch.zeros(weight_int.size(1), len(o_nig)*len(o_njg)).cuda()
  
   
### SFP accumulation ###
for ni in o_nig:
    for nj in o_njg:
        for ki in kig:
            for kj in kjg:
                for ic_tile in ic_tileg:    
                    for oc_tile in oc_tileg:   
                        out[oc_tile*array_size:(oc_tile+1)*array_size, ni*len(o_nig)+nj] += psum[ic_tile, oc_tile, :, int(math.sqrt(len(p_nijg)))*(ni+ki) + (nj+kj), len(kig)*ki+kj]

In [13]:
output_int[0,:,:,:]

tensor([[[-1.0000e+01, -9.6000e+01,  8.9000e+01,  ...,  8.6000e+01,
          -1.2300e+02,  2.6300e+02],
         [-1.3100e+02, -1.8000e+02,  4.4000e+01,  ...,  4.5000e+01,
          -2.1400e+02,  3.8200e+02],
         [-8.9000e+01, -2.9500e+02,  1.0000e+00,  ..., -3.8000e+01,
          -3.0100e+02,  4.4100e+02],
         ...,
         [-5.9999e+00,  7.2900e+02,  3.8300e+02,  ...,  4.9300e+02,
           6.4500e+02, -3.2900e+02],
         [ 5.7000e+01,  5.1400e+02,  3.4900e+02,  ...,  6.6700e+02,
           2.1800e+02,  1.1700e+02],
         [ 6.3000e+01,  3.0900e+02,  2.3800e+02,  ...,  3.7500e+02,
           7.9000e+01,  2.1400e+02]],

        [[-1.5700e+02, -1.3200e+02,  1.1300e+02,  ...,  2.5100e+02,
           2.4900e+02,  2.6500e+02],
         [-2.5500e+02, -3.3800e+02, -1.1000e+02,  ...,  7.6000e+01,
           1.7000e+02,  3.0700e+02],
         [-2.4300e+02, -4.2100e+02, -2.7000e+02,  ..., -1.4600e+02,
           7.4000e+01,  3.0400e+02],
         ...,
         [ 3.8800e+02,  4

In [14]:
difference = (out - torch.reshape(output_int[0,:,:,:], (out.size(0), -1)))
print(difference.sum())

tensor(0.2913, device='cuda:0', grad_fn=<SumBackward0>)


In [15]:
a_tile.shape

torch.Size([4, 4, 16, 1156])

In [16]:
psum.shape

torch.Size([4, 4, 16, 1156, 9])

In [17]:
out.shape

torch.Size([64, 1024])